### Goal 1:

Create a context manager that you can use to produce the data from each file in a named tuple with field names corresponding to the header row field names (use the `csv` module's `reader` function to help with parsing the data).

Besides, the context manager should be generic in the sense that it should just need the file name, no other configuration or hardcoded functionality is required.

In addition, the context manager should produce lazy iterators.

In [1]:
import csv
from itertools import islice
from collections import namedtuple

In [2]:
class ContextManager:
    
    def __init__(self, file_path):
        self.file_path = file_path
        
    def __enter__(self):
        self.obj = open(self.file_path)
        
        header = self.obj.readline()
        self.delimiter = csv.Sniffer().sniff(header).delimiter
        self.header = header.strip('\n').split(self.delimiter)
        self.nt = namedtuple('Data', self.header)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.obj.close()
        if exc_type:
            print('Exception: {0}, {1}'.format(exc_type, exc_val, exc_tb))
        return False
    
    def __iter__(self):
        return self
    
    def __next__(self):
        return self.nt(*self.obj.readline().strip('\n').split(self.delimiter))
        

In [3]:
with ContextManager('./cars.csv') as f:
    for row in islice(f, 5):
        print(row)
        print('-'*127)

Data(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')
-------------------------------------------------------------------------------------------------------------------------------
Data(Car='Buick Skylark 320', MPG='15.0', Cylinders='8', Displacement='350.0', Horsepower='165.0', Weight='3693.', Acceleration='11.5', Model='70', Origin='US')
-------------------------------------------------------------------------------------------------------------------------------
Data(Car='Plymouth Satellite', MPG='18.0', Cylinders='8', Displacement='318.0', Horsepower='150.0', Weight='3436.', Acceleration='11.0', Model='70', Origin='US')
-------------------------------------------------------------------------------------------------------------------------------
Data(Car='AMC Rebel SST', MPG='16.0', Cylinders='8', Displacement='304.0', Horsepower='150.0', Weight='3433.', Acceleration=

In [4]:
with ContextManager('./personal_info.csv') as f:
    for row in islice(f, 5):
        print(row)
        print('-'*127)

Data(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
-------------------------------------------------------------------------------------------------------------------------------
Data(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
-------------------------------------------------------------------------------------------------------------------------------
Data(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
-------------------------------------------------------------------------------------------------------------------------------
Data(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi')
-------------------------------------------------------------------------------------------------------------------------------
Data(ssn='104-84-7144', first_name='Claudianus', last_name='Brixey', gend

---

### Goal 2:

Use a generator function and the `contextlib` `contextmanager` decorator to achieve the same function in goal 1.

In [5]:
from contextlib import contextmanager

In [6]:
@contextmanager
def parsed_data(file_path):
    
    def parsed_data_iter(data_iter, nt):
        for row in data_iter:
            yield nt(*row)
            
    f = open(file_path)
    try:
        header = f.readline()
        dialect = csv.Sniffer().sniff(header)
        reader = csv.reader(f, dialect)
        header = header.strip('\n').split(dialect.delimiter)
        nt = namedtuple('Data', header)
        
        yield parsed_data_iter(reader, nt)
        
    finally:
        f.close()

In [7]:
with parsed_data('./cars.csv') as f:
    for row in islice(f, 5):
        print(row)
        print('-'*127)

Data(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')
-------------------------------------------------------------------------------------------------------------------------------
Data(Car='Buick Skylark 320', MPG='15.0', Cylinders='8', Displacement='350.0', Horsepower='165.0', Weight='3693.', Acceleration='11.5', Model='70', Origin='US')
-------------------------------------------------------------------------------------------------------------------------------
Data(Car='Plymouth Satellite', MPG='18.0', Cylinders='8', Displacement='318.0', Horsepower='150.0', Weight='3436.', Acceleration='11.0', Model='70', Origin='US')
-------------------------------------------------------------------------------------------------------------------------------
Data(Car='AMC Rebel SST', MPG='16.0', Cylinders='8', Displacement='304.0', Horsepower='150.0', Weight='3433.', Acceleration=